<a href="https://colab.research.google.com/github/alcliberal/multirotor_sizing_isae_coa_2021/blob/main/05b_MotorSelection-Student.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img src="https://github.com/alcliberal/multirotor_sizing_isae_coa_2021/blob/main/pictures/logo_sizinglab.png?raw=1" style="float:right; max-width: 60px; display: inline" alt="SizingLab" /></a>

# Motor selection
*Written by Marc Budinger (INSA Toulouse) and Scott Delbecq (ISAE-SUPAERO), Toulouse, France.*

## Design graph 

The following diagram represents the design graph of the motor’s selection. The mean speed/thrust (Ωmoy & Tmoy), the max speed/thrust (Ωmax & Tmax) and the battery voltage are assumed to be known here.

![DesignGraph](https://github.com/alcliberal/multirotor_sizing_isae_coa_2021/blob/main/pictures/MotorDesignGraph.png?raw=1)

> **Questions:**
* Give the 2 main sizing problems you are able to detect here.
* Propose one or multiple solutions (which can request equation manipulation, addition of design variables, addition of constraints) 
* Orientate the arrows and write equations order, inputs/outputs at each step of this part of sizing procedure, additional constraints


### Sizing code and optimization

> Exercice: propose a sizing code for the selection of a motor.


In [2]:
# Specifications

# Reference parameters for scaling laws
# Motor reference
# Ref : AXI 5325/16 GOLD LINE
T_nom_mot_ref = 2.32  # [N.m] rated torque
T_max_mot_ref = 85./70.*T_nom_mot_ref # [N.m] max torque
R_mot_ref = 0.03  # [Ohm] resistance
M_mot_ref = 0.575 # [kg] mass
K_mot_ref = 0.03 # [N.m/A] torque coefficient
T_mot_fr_ref = 0.03 # [N.m] friction torque (zero load, nominal speed)

# Assumption
T_pro_to=0.5#[N.m] Propeller Torque during takeoff
Omega_pro_to=400.0#[rad/s] Propeller speed during takeoff
T_pro_hov=1.0#[N.m] Propeller Torque during hover
Omega_pro_hov=0.22#[rad/s] Propeller speed during hover
U_bat_est= 14.0#[V] Battery voltage value (estimation)


In [3]:
#Design variables
k_mot = 1#over sizing coefficient on the motor torque
k_speed_mot = 1  # over sizing coefficient on the motor speed

#Equations:
#-----
T_nom_mot = k_mot * T_pro_hov  # [N.m] Motor nominal torque per propeller
M_mot = M_mot_ref * (T_nom_mot/T_nom_mot_ref)**(3/3.5)  # [kg] Motor mass (scaling law)
# Selection with take-off speed
K_mot = U_bat_est/Omega_pro_to/k_speed_mot # [N.m/A] or [V/(rad/s)] Kt motor
R_mot = R_mot_ref * (K_mot/K_mot_ref)**2 * (T_nom_mot/T_nom_mot_ref)**(-5/3.5)  # [Ohm] motor resistance (scaling law)

T_mot_fr = T_mot_fr_ref * (T_nom_mot/T_nom_mot_ref)**(3/3.5) # [N.m] Friction torque (scaling law)

T_max_mot = T_max_mot_ref*(T_nom_mot/T_nom_mot_ref)

# Hover current and voltage
I_mot_hov = (T_nom_mot+T_mot_fr) * R_mot # [I] Current of the motor per propeller
U_mot_hov = R_mot * I_mot_hov + K_mot * Omega_pro_hov # [V] Voltage of the motor per propeller
P_el_mot_hov = I_mot_hov * U_mot_hov  # [W] Hover : electrical power
# Takeoff current and voltage
I_mot_to = (T_max_mot+T_mot_fr) * R_mot # [I] Current of the motor per propeller
U_mot_to = R_mot * I_mot_to + K_mot * Omega_pro_to # [V] Voltage of the motor per propeller
P_el_mot_to = I_mot_to * U_mot_to # [W] Takeoff : electrical power

In [ ]:
k